In [4]:
##################################################################################
#Wczytanie potrzebnych bibliotek
import snap7
import matplotlib.pyplot as plt
import pandas as pd
import schedule
import time
import numpy
import threading
import tkinter as tk
import queue


##################################################################################
"""""
# Inicjalizacja kolejki priorytetowej dla wątków
priority_queue = queue.PriorityQueue()

# Dodanie priorytetów dla wątków
EXTERNAL_DATA_PRIORITY = 1
EXTERNAL_DATA_2_PRIORITY = 2


##################################################################################
#Monitoring danych co jedną sekundę
#W przypadku wykrycia zmiany, funkcja main() będzie wywołana
"""""


#Utworzenie klienta 
plc = snap7.client.Client()
    #Połączenie z sterownikiem PLC za pomocą adresu IP (Slot i Rack)
plc.connect('169.254.90.50', 0, 1)

"""""
#I wątek
def monitor_external_data():
        stara = main()
        nowa, _ = Get_Real()
        check_for_change(stara, nowa)

def check_for_change(stara, nowa):
    PLC_COM()
    nowa, _ = Get_Real()
    if nowa != stara:
        main()
    root.after(1000, check_for_change, nowa, nowa)


#II wątek
def monitor_external_data_2():
    setpoint, nowe_Wy = Get_Real()
    check_for_change_2(nowe_Wy, setpoint)
        
def check_for_change_2(nowe_Wy, setpoint, stabilizacja=0, delta=1):
    Wyjscie_zapis_srawdz()
    PLC_COM()
    _, nowe_Wy = Get_Real()
    if setpoint - delta <= nowe_Wy <= setpoint + delta:
        stabilizacja += 1
        if stabilizacja >= 10:  # Sprawdzamy, czy stabilizacja trwa przez co najmniej 10 sekund
            Wyjscie_odczyt()
            stabilizacja = 0  # Resetujemy licznik stabilizacji
    else:
        stabilizacja = 0  # Resetujemy licznik stabilizacji
    root.after(1000, check_for_change_2, nowe_Wy, setpoint, stabilizacja, delta)

    """""
def monitor_external_data():
    stara = main()
    setpoint, _ = Get_Real()
    #nowa, _ = Get_Real()
    check_for_change(stara, setpoint)


def check_for_change(stara, setpoint, stabilizacja=0, delta=1):
    Wyjscie_zapis_srawdz()
    PLC_COM()
    setpoint, nowe_Wy = Get_Real()
    if setpoint != stara:
        main()
    if setpoint == stara:
        if setpoint - delta <= nowe_Wy <= setpoint + delta:
            stabilizacja += 1
            if stabilizacja >= 10:  # Sprawdzamy, czy stabilizacja trwa przez co najmniej 10 sekund
                Wyjscie_odczyt()
                stabilizacja = 0  # Resetujemy licznik stabilizacji
        else:
            stabilizacja = 0  # Resetujemy licznik stabilizacji
    root.after(1000, check_for_change, stara, setpoint, stabilizacja, delta)
    
###################################################################################
#Funkcje do wczytywania danych i ich analizy
        
def PLC_COM():

    #Dane dotyczące zmiennych, które chcemy otrzymać
    DB_number_1 = 1
    Offset_number_1 = 0
    Length_1 = 4

    # Zwrot wartosci wyjscia
    DB_number_2 = 2
    Offset_number_2 = 0
    Length_2 = 4

    # Zwrot wartosci error
    # DB_number_3 = ..
    # Offset_number_3 = ..
    # Length_3 = ..
    # return plc.db_read(DB_number_3,Offset_number_3,Length_3)

    return plc.db_read(DB_number_1,Offset_number_1,Length_1), plc.db_read(DB_number_2,Offset_number_2,Length_2)
 



def Get_Real():
    #numer DB, Offset numer, dlugosc danych w bajtach
    #...data_2, data3 = PLC_COM()
    data_1, data_2 = PLC_COM()
    Zmienna_1 = snap7.util.get_real(data_1, 0)
    Zmienna_2 = snap7.util.get_real(data_2, 0)
    # Zmienna_3 = snap7.util.get_real(data_3, 0)
    # return Zmienna_2, Zmienna_3
    return Zmienna_1, Zmienna_2

#Funkcja obliczająca średnią
def średnia_wynik(size, y):  
    num = 0
    dod = 0
    war_sr = 0
    while num < size:
        for i in y:
            dod=dod+i
            num+=1
            war_sr = dod/size
    print("Wartość średnia: ", war_sr)
    return war_sr

#Poczytac o tym jeszcze

#def calculate_ise(errors):
    """
    Function to calculate the Integral of Squared Error (ISE) given a list of errors.
    
    Args:
    errors (list): List of error values over time.
    
    Returns:
    ise (float): Integral of Squared Error.
    """

    """"
    ise = 0
    for error in errors:
        ise += error ** 2
    return ise

error_values = [1, 2, 3, 4, 5]  # Przykładowe wartości błędów w czasie
total_ise = calculate_ise(error_values)
print("Integral of Squared Error:", total_ise)
"""



def main():
    
    Setpoint, _ = Get_Real()

    # Dane do zapisania do pliku Excel
    data = {
        'Y': [Setpoint],
    }

    #Próba odczytu istniejących danych z excela
    try:
        df_existing = pd.read_excel("liczby.xlsx") 
    except FileNotFoundError:
        df_existing = pd.DataFrame()

    #Stworzenie nowej ramki danych
    df_new = pd.DataFrame(data)

    #Tworzenie wartości X odpowiadających kolejnym liczbom naturalnym
    start_index = len(df_existing) + 1  #Indeks startowy dla X
    end_index = start_index + len(df_new)  #Indeks końcowy dla X
    x_values = list(range(start_index, end_index))

    #Dodanie kolumny X do ramki danych
    df_new['X'] = x_values

    #Połączenie nowych i starych danych tak, aby sie nie nadpisywały
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    df_combined.to_excel("liczby.xlsx", index=False)

    #Odczyt z excela
    df = pd.read_excel(r"C:\Users\kamil.mika\Desktop\DataAnalysisPLC\liczby.xlsx")

    #Przykładowe dane
    kolumna_x = 'X'
    kolumna_y = 'Y'

    ####### Dla danych z excela
    x = df[kolumna_x]
    y = df[kolumna_y]

    ########Wartości surowe
    #######x = [1, 2, 3, 4, 5]
    #######y = [1.5, 10, 6, 12, 10]
    size = len(y)

    #Znalezenie wartości maks i min (Dla wartości z excela)
    indeks_max = y.idxmax()
    indeks_min = y.idxmin()

    ########Dla wartości wpisanych
    #######indeks_max = y.index(max(y))
    #######indeks_min = y.index(min(y))

    maksymalny_punkt = (x[indeks_max], y[indeks_max])
    minimalny_punkt = (x[indeks_min], y[indeks_min])
    print("Wartość maksymalna: ", y[indeks_max])
    print("Wartość minimalna: ", y[indeks_min])

    #Wartość średnia
    wynik_srednia = str(średnia_wynik(size, y))

    #Wykres przedstawiający przebieg wartości setpointu
    plt.figure(1)
    plt.plot(x, y, 'red', linewidth = 2)
    plt.title('Przebieg wartości zadanej (Setpoint)')
    plt.xlabel('Oś X')
    plt.ylabel('Oś Y')

    #Wyswietlanie wartosci na wykresie (punkty)
    plt.figure(2)
    plt.scatter(x, y)
    plt.scatter(*maksymalny_punkt,color='red', label='Punkt maksymalny')
    plt.scatter(*minimalny_punkt,color='violet', label='Punkt minimalny')
    plt.title('Zbiór wyników')
    plt.xlabel('Oś X')
    plt.ylabel('Oś Y')
    plt.legend()

    #Wyświetlanie wykresów
    plt.show()

    #######Automatyczne zamykanie
    #######plt.show(block=False)
    #######plt.pause(3)
    #######plt.close()

    #Zapis do pliku
    f = open("Wyniki.txt", "w")
    f.write("Wartość średnia: " + wynik_srednia+'\n')
    f.write("Wartość maksymalna: " + str(y[indeks_max])+'\n')
    f.write("Wartość minimalna: " + str(y[indeks_min])+'\n')
    f.close()
    return Setpoint


def Wyjscie_zapis_srawdz():
    _, wyjscie = Get_Real()

     # Dane do zapisania do pliku Excel
    data_output = {
        'Y': [wyjscie],
    } 

    #Próba odczytu istniejących danych z excela
    try:
        df_existing = pd.read_excel("Wyjscie.xlsx") 
    except FileNotFoundError:
        df_existing = pd.DataFrame()

    #Stworzenie nowej ramki danych
    df_new = pd.DataFrame(data_output)

    #Tworzenie wartości X odpowiadających kolejnym liczbom naturalnym
    start_index = len(df_existing) + 1  #Indeks startowy dla X
    end_index = start_index + len(df_new)  #Indeks końcowy dla X
    x_values = list(range(start_index, end_index))

    #Dodanie kolumny X do ramki danych
    df_new['X'] = x_values

    #Połączenie nowych i starych danych tak, aby sie nie nadpisywały
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    df_combined.to_excel("Wyjscie.xlsx", index=False)
    time.sleep(0.5)

def Wyjscie_odczyt():
    print("Jestes w domu")



###################################################################################
"""""
    # Funkcje do obsługi kolejki priorytetowej wątków
def process_queue():
    while not priority_queue.empty():
        priority, task = priority_queue.get()
        task()

def add_task_to_queue(priority, task):
    priority_queue.put((priority, task))
    # Uruchomienie funkcji process_queue po dodaniu zadania do kolejki
    root.after(0, process_queue)
"""""



root = tk.Tk()
external_data_monitor_thread_1 = threading.Thread(target=monitor_external_data)
external_data_monitor_thread_1.start()

#add_task_to_queue(EXTERNAL_DATA_PRIORITY, monitor_external_data)
#add_task_to_queue(EXTERNAL_DATA_2_PRIORITY, monitor_external_data_2)

#external_data_monitor_thread_2 = threading.Thread(target=monitor_external_data_2)
#external_data_monitor_thread_2.start()
root.mainloop()

    

#Funkcja after() umożliwia planowanie wywołania funkcji po określonym czasie, 
#co może być przydatne w przypadku cyklicznego sprawdzania zmiany danych.
#W tym kodzie funkcja check_for_change() jest wywoływana cyklicznie co sekundę za pomocą funkcji after().
#Funkcja ta sprawdza zmiany w danych i wywołuje main() w przypadku wykrycia zmiany. 
#Ten mechanizm pozwala na sprawdzanie zmian w tle, bez konieczności używania bezpośrednio głównej pętli programu








10.0
